
# Hate Speech Span Detection and Categorization


In [ ]:

# --- Configuration ---
# Fill these paths and hyperparameters before running.
CONFIG = {
    # Task: choose one of ["categorization", "detection"]
    "task": "categorization",

    # Data: CSVs with columns: Tweet_id, tokens(list), tags(list)
    "train_set": "train.csv",
    "test_set": "test.csv",
    "non_hateful": "non_hateful.csv",

    # Model & Training
    "model_name": "dbmdz/bert-base-turkish-cased",
    "learning_rate": 5e-5,
    "batch_size": 4,
    "epochs": 10,
    "warmup_ratio": 0.1,
    "early_stop_patience": 3,
    "weight_decay": 0.0,
    "eval_train_split_ratio": 0.1,  # fraction of train used as eval
    "test_sample_size": 300,         # held-out sample from test_set (0 = use full test file)
    "random_state": 42,
    "log_steps": 100,

    # Output dir (Trainer output)
    "output_dir": "train"
}
print("Loaded CONFIG. Edit paths and hyperparameters above as needed.")

In [ ]:

# --- Imports ---
import os
from ast import literal_eval
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
import evaluate
from transformers import (
    AutoTokenizer,
    DataCollatorForTokenClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
)

# --- Label Maps ---
CATEGORIZATION_ID2LABEL = {
    0: "O",
    1: "I-Exclusive/Discriminatory Discourse",
    2: "I-Exaggeration; Generalization; Attribution; Distortion",
    3: "I-Threat of Enmity; War; Attack; Murder; or Harm",
    4: "I-Symbolization",
    5: "I-Swearing; Insult; Defamation; Dehumanization",
}
CATEGORIZATION_LABEL2ID = {v: k for k, v in CATEGORIZATION_ID2LABEL.items()}

DETECTION_ID2LABEL = {
    0: "O",
    1: "B-HATE",
    2: "I-HATE",
}
DETECTION_LABEL2ID = {v: k for k, v in DETECTION_ID2LABEL.items()}
print("Imports complete and label maps ready.")

In [ ]:

# --- Helper Functions ---
def load_dataframes(
    train_set_path: str,
    test_set_path: str,
    non_hateful_path: str,
    test_sample_size: int,
    random_state: int = 42,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Returns (df_train_set_merged_with_non_hateful, df_test)."""
    df_train_set = pd.read_csv(train_set_path)
    df_test_set = pd.read_csv(test_set_path)
    df_not_spans = pd.read_csv(non_hateful_path)

    for df in (df_test_set, df_train_set, df_not_spans):
        df["tokens"] = df["tokens"].apply(literal_eval)
        df["tags"] = df["tags"].apply(literal_eval)
        df["Tweet_id"] = df["Tweet_id"]

    df_train_set_merged_with_non_hateful = pd.concat([df_train_set, df_not_spans], ignore_index=True)

    if test_sample_size > 0:
        df_test = df_test_set.sample(test_sample_size, random_state=random_state)
    else:
        df_test = df_test_set.copy()

    exclude_ids = set(df_test["Tweet_id"].tolist())
    df_train_set_merged_with_non_hateful = df_train_set_merged_with_non_hateful[
        ~df_train_set_merged_with_non_hateful["Tweet_id"].isin(exclude_ids)
    ]

    return df_train_set_merged_with_non_hateful, df_test


def make_hf_datasets(df_train_set_merged_with_non_hateful: pd.DataFrame, df_test: pd.DataFrame) -> DatasetDict:
    train_dataset = Dataset.from_pandas(df_train_set_merged_with_non_hateful)
    test_dataset = Dataset.from_pandas(df_test)
    ds = DatasetDict({"train": train_dataset, "test": test_dataset})
    return ds


def build_tokenizer(model_name: str):
    tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
    return tokenizer


def align_labels_with_tokens(examples, tokenizer, label_list: List[str]):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


def build_metrics(id2label: Dict[int, str]):
    seqeval = evaluate.load("seqeval")
    label_list = list(id2label.values())

    def compute_metrics(p):
        predictions, labels = p
        predictions = np.argmax(predictions, axis=2)

        true_predictions = [
            [label_list[p_i] for (p_i, l_i) in zip(prediction, label) if l_i != -100]
            for prediction, label in zip(predictions, labels)
        ]
        true_labels = [
            [label_list[l_i] for (p_i, l_i) in zip(prediction, label) if l_i != -100]
            for prediction, label in zip(predictions, labels)
        ]

        results = seqeval.compute(predictions=true_predictions, references=true_labels)
        return {
            "precision": results.get("overall_precision", 0.0),
            "recall": results.get("overall_recall", 0.0),
            "f1": results.get("overall_f1", 0.0),
            "accuracy": results.get("overall_accuracy", 0.0),
        }

    return compute_metrics


def build_trainer(
    tokenized_dataset: DatasetDict,
    model_name: str,
    id2label: Dict[int, str],
    label2id: Dict[str, int],
    learning_rate: float,
    batch_size: int,
    num_epochs: int,
    warmup_ratio: float,
    weight_decay: float,
    early_stop_patience: int,
    logging_steps: int = 100,
    eval_train_split_ratio: float = 0.1,
    seed: int = 42,
):
    num_labels = len(id2label)

    model = AutoModelForTokenClassification.from_pretrained(
        model_name, num_labels=num_labels, id2label=id2label, label2id=label2id
    )
    tokenizer = build_tokenizer(model_name)
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
    compute_metrics = build_metrics(id2label)

    training_args = TrainingArguments(
        output_dir=CONFIG["output_dir"],
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        metric_for_best_model="eval_f1",
        load_best_model_at_end=True,
        greater_is_better=True,
        num_train_epochs=num_epochs,
        evaluation_strategy="epoch",   # FIX: correct HF arg (instead of eval_strategy)
        save_strategy="epoch",
        logging_dir=os.path.join(CONFIG["output_dir"], "logs"),
        logging_steps=logging_steps,
        save_total_limit=1,
        push_to_hub=False,
        warmup_ratio=warmup_ratio,
        weight_decay=weight_decay,
    )

    split_ds = tokenized_dataset["train"].train_test_split(test_size=eval_train_split_ratio, seed=seed)
    train_dataset = split_ds["train"]
    eval_dataset = split_ds["test"]

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=early_stop_patience)],
    )

    return trainer, tokenizer
print("Helper functions defined.")

In [ ]:

# --- Build Datasets ---
task = CONFIG["task"]
if task == "categorization":
    id2label = CATEGORIZATION_ID2LABEL
    label2id = CATEGORIZATION_LABEL2ID
else:
    id2label = DETECTION_ID2LABEL
    label2id = DETECTION_LABEL2ID

df_train_set_merged_with_non_hateful, df_test = load_dataframes(
    CONFIG["train_set"],
    CONFIG["test_set"],
    CONFIG["non_hateful"],
    CONFIG["test_sample_size"],
    CONFIG["random_state"],
)
ds = make_hf_datasets(df_train_set_merged_with_non_hateful, df_test)

tokenizer = build_tokenizer(CONFIG["model_name"])

def _map_fn(examples):
    return align_labels_with_tokens(examples, tokenizer, list(id2label.values()))

tokenized_dataset = ds.map(_map_fn, batched=True)
print(tokenized_dataset)

In [ ]:

# --- Train ---
trainer, tokenizer = build_trainer(
    tokenized_dataset=tokenized_dataset,
    model_name=CONFIG["model_name"],
    id2label=id2label,
    label2id=label2id,
    learning_rate=CONFIG["learning_rate"],
    batch_size=CONFIG["batch_size"],
    num_epochs=CONFIG["epochs"],
    warmup_ratio=CONFIG["warmup_ratio"],
    weight_decay=CONFIG["weight_decay"],
    early_stop_patience=CONFIG["early_stop_patience"],
    logging_steps=CONFIG["log_steps"],
    eval_train_split_ratio=CONFIG["eval_train_split_ratio"],
    seed=CONFIG["random_state"]
)

# Uncomment to run training:
# trainer.train()
print("Trainer is ready. Uncomment trainer.train() to start training.")

In [ ]:

# --- Evaluate on held-out test set ---
# Uncomment after training:
# metrics = trainer.evaluate(tokenized_dataset["test"])
# print(f"[{task.upper()}] Test metrics:", metrics)

# predictions, label_ids, metrics = trainer.predict(tokenized_dataset["test"])
# print(f"[{task.upper()}] Test metrics from predict:", metrics)